In [27]:
#Takes a clinical note and generates a valid FHIR bundle of the most critical pieces

import re
from chromadb import api
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableSequence  # Import RunnableSequence
import os
from langchain_core.output_parsers import JsonOutputParser
import shutil
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import logging
from langchain.globals import set_verbose
set_verbose(True)


In [25]:
# Configure logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("fhir-transformer")
load_dotenv()

True

In [ ]:
def setup():
    google_api_key = os.getenv("GOOGLE_API_KEY")
    if not google_api_key:
        raise ValueError("GOOGLE_API_KEY environment variable is not found.")
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17", temperature=0.0, google_api_key=google_api_key)
    return llm
llm = setup()

# Make the Patient Resource

##### Note: for US IG mandatory elemets: identifier, resourcetype, meta.profile, active, name, gender, birthDate, race

In [36]:

template = """<System> You are a clinical NLP system and clinicical informatics expert that specializes in medical ontologies like Snomed CT, ICD-10, LOINC, and CPT.

<Task>Extract the following patient information from this clinical note: {note}

1. name
2. age
3. gender
4. unique identifier
5. race
6. contact information
7. active status

If a piece of information is not present, return Null.

<Format Instructions>
{format_instructions}
"""

In [34]:
# #example of a compliant Patient resource
# {
#   "resourceType": "Patient",
#   "id": "12345",
#   "meta": {
#     "profile": [
#       "http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient"
#     ]
#   },
#   "identifier": [
#     {
#       "system": "http://hospital.smarthealthit.org",
#       "value": "MRN123456"
#     }
#   ],
#   "active": true,
#   "name": [
#     {
#       "family": "Doe",
#       "given": ["John"]
#     }
#   ],
#   "gender": "male",
#   "birthDate": "1980-01-01",
#   "extension": [
#     {
#       "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-race",
#       "valueCodeableConcept": {
#         "coding": [
#           {
#             "system": "http://hl7.org/fhir/v3/Race",
#             "code": "2106-3",
#             "display": "White"
#           }
#         ],
#         "text": "White"
#       }
#     },
#     {
#       "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity",
#       "valueCodeableConcept": {
#         "coding": [
#           {
#             "system": "http://hl7.org/fhir/v3/Ethnicity",
#             "code": "2186-5",
#             "display": "Not Hispanic or Latino"
#           }
#         ],
#         "text": "Not Hispanic or Latino"
#       }
#     }
#   ],
#   "telecom": [
#     {
#       "system": "phone",
#       "value": "555-123-4567",
#       "use": "home"
#     }
#   ],
#   "address": [
#     {
#       "line": ["123 Main St"],
#       "city": "Metropolis",
#       "state": "NY",
#       "postalCode": "10001",
#       "country": "USA"
#     }
#   ]
# }

In [37]:
#let's try a inpatient medicine discharge summary for a CHF stay
prompt = PromptTemplate.from_template(template)

try:
    note = open("../clinical_notes_repository/note_1_medicine_CHF.txt").read()
    parser = JsonOutputParser()
    format_instructions = parser.get_format_instructions()
    chain = prompt | llm | parser
    response = chain.invoke({'note': note,'format_instructions': format_instructions})
    print(response)
except Exception as e:
    print(f"Error extracting patient data: {e}")

{'name': None, 'age': 80, 'gender': 'F', 'unique_identifier': None, 'race': None, 'contact_information': None, 'active_status': None}


# Make the Condition Resource